# Song Name Hot or Not

* Goal: To generate candidates for the best or worst song name using visualizations
* Method: Using the names of each song in the db, extract the most frequent bigrams. Show the highest count relationships in a chord diagram.

## Imports

In [1]:
# base
import pandas as pd
import numpy as np
import nltk
from nltk import bigrams
import itertools
import pandas as pd
import re

# Chord/Shahin
from chord import Chord

# hv
import holoviews as hv
from holoviews import opts, dim

## Utility Functions

In [2]:
# remove stopwords
# input: df, column name containing words
# output: df with stopwords removed

def rm_stopwords(df, name):
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    sw = stopwords.words('english')
    realwords = df[~df[name].isin(sw)]
    return realwords

In [3]:
# create a matrix for word in each song
# input: df with column of words, one per row, multiple duplicates of song title
# df 'realwords'
# column 'title'

def longtable_to_listoflist(realwords):

    finallist= []
    wordlist = []

    uniq_title = realwords.title.unique()

    for song in uniq_title:
        wordlist = []
        print(song)
        subset = realwords[realwords['title']==song]
        for word in subset['word']:
            wordlist.append(word)
        finallist.append(wordlist)

    print(finallist)
    return finallist

In [4]:
# generate chord chart to html
# input: matrix, names, output name
# output: html file saved to same directory

def gen_html(matrix, names, output):
    Chord(matrix, names, padding=0.05, width=1200, wrap_labels=False).to_html(output)

In [5]:
# generate matrix for chord/shahin
# limit is min co-occur #

def gen_chord(inlist, limit):
    data = list(itertools.chain.from_iterable(inlist))
    matrix, bigram, vocab_index = generate_co_occurrence_matrix(data)

    data_matrix = pd.DataFrame(matrix, index=vocab_index, columns=vocab_index)
    
    data_matrix = rematrix(data_matrix, limit)
    
    outmatrix = data_matrix.to_numpy()
    outmatrix = outmatrix.tolist()
#     outmatrix = matrix.tolist()
    names = list(data_matrix.index) 

#     print(matrix)
#     print(type(matrix))
#     print(data_matrix)
#     print(type(names))
#     print(names)
    return outmatrix, data_matrix, bigram, names

In [6]:
# build co-occurrence matrix
# input: list of lists
# output: outmatrix

def generate_co_occurrence_matrix(corpus):
    vocab = set(corpus)
    print('vocab len: ' + str(len(vocab)))
    vocab = list(vocab)
    vocab_index = {word: i for i, word in enumerate(vocab)}

    # Create bigrams from all words in corpus
    bi_grams = list(bigrams(corpus))

    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    bigram_freqraw = nltk.FreqDist(bi_grams).most_common(len(bi_grams))
    bigram_freq = [bigram for bigram in bigram_freqraw if bigram[1] > 1]

    # Initialise co-occurrence matrix
    # co_occurrence_matrix[current][previous]
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))

    # Loop through the bigrams taking the current and previous word,
    # and the number of occurrences of the bigram.
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]
        pos_current = vocab_index[current]
        pos_previous = vocab_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count
    co_occurrence_matrix = np.matrix(co_occurrence_matrix)

    # return the matrix and the index
    return co_occurrence_matrix, bigram_freq, vocab_index

In [7]:
# strip cooccurence matrix of low value weights per limit

def rematrix(df, limit):
    for idx in df.index:
        rowsum = df.loc[idx,:].sum(axis=0)
        colsum = df[idx].sum(axis=0)
#         print('Idx: ' + idx + '; sum of row: ' + str(rowsum))
        if rowsum < limit and colsum < limit:
            df.drop([idx], axis = 0, inplace=True)
            df.drop([idx], axis = 1, inplace=True)
#             print('dropping: ' + idx)
    print('Final shape: ')
    print(df.shape)
    return df

In [8]:
# remove parentheses and contents from song titles (or any string input)

def Clean_titles(title): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times
    if re.search('\(.*', str(title)):
        # return the cleaned name 
        return re.sub(r'\([^)]*\)', '', str(title)) # FORCE TO STRING in case of floats
    else: 
        # if clean up needed return the same name 
        return title 

## Load cleaned data

In [9]:
deduped = pd.read_csv(r"../Py code/titles_clean.zip")
deduped.head()

,Unnamed: 0,word,count,track_id,song_id,artist_id,title,artist_name,duration,year,artist_hotttnesss
0,0,i,6,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
1,68,i,10,TRAAABD128F429CF47,SOCIWDW12A8C13D406,ARMJAGH1187FB546F3,Soul Deep,The Box Tops,148.03546,1969,0.417500
2,142,i,28,TRAAAED128E0783FAB,SOXZYWX12A6310ED0C,ARC1IHZ1187FB4E920,It's About Time,Jamie Cullum,246.98730,0,0.562061
3,262,i,5,TRAAAEF128F4273421,SONHOTT12A8C13493C,AR7G5I41187FB4CE6C,Something Girls,Adam Ant,233.40363,1982,0.454231
4,321,i,4,TRAAAEW128F42930C0,SODZYPO12A8C13A91E,AR1C2IX1187B99BF74,Burn My Body,Broken Spindles,177.99791,0,0.378545


### Set parameters

In [12]:
# set thresholds

hot_thresh = 0.75 # recommend > 0.7
not_thresh = 0.25
limit = 4

# caution INPLACE
dropcols = ['track_id', 'song_id', 'artist_id', 'duration']

In [13]:
# filter down to hotties
hotties = deduped[deduped['artist_hotttnesss'] > hot_thresh]
hotties.drop(columns=dropcols, inplace=True)
unique_hotties = hotties.title.unique()
print(str(len(unique_hotties)) + ' unique songs with hotness greater than ' + str(hot_thresh))
print(unique_hotties)

1739 unique songs with hotness greater than 0.75
["Let's Get It Started" 'Low [feat. T-Pain] [Travis Barker Remix]'
 'Kite Live from Sydney' ... 'I Just Wanna Know' 'The Calm' 'Solo Dolo ']


C:\Users\Wx\.conda\envs\dv\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
# filter down to NOTties, doing yourself no favors with naming convention here
notties = deduped[(deduped['artist_hotttnesss'] < not_thresh) & (deduped['artist_hotttnesss'] > 0)]
notties.drop(columns=dropcols, inplace=True)
unique_notties = notties.title.unique()
print(str(len(unique_notties)) + ' unique songs with hotness less than ' + str(not_thresh))
print(unique_notties)

1613 unique songs with hotness less than 0.25
['Next Time' 'Überlebensgroß' 'Non So Perchè ' ... 'Chills'
 'Yo se que te amare ' 'Foggy Mountain Special']


### Build the title data

In [15]:
# create a matrix for word in each song
# input: list of song names
# output: list of lists, each song name split into component words

hotlist = []
notlist = []

for song in unique_hotties:
    subset = song.split()
    hotlist.append(subset)
    
for song in unique_notties:
    subset = song.split()
    notlist.append(subset)
        
print('Hotlist: ')
# print(hotlist)
print('\nNotlist: ')
# print(notlist)

Hotlist: 

Notlist: 


In [16]:
# hot_mat, hot_data, hot_bigrams, hot_names - explain each one
hot_mat, hot_data, hot_bigrams, hot_names = gen_chord(hotlist, limit)

vocab len: 2093
Final shape: 
(80, 80)


In [17]:
not_mat, not_data, not_bigrams, not_names = gen_chord(notlist, limit)

vocab len: 2388
Final shape: 
(45, 45)


### Final cleanup on hot_mat and hot_names

In [20]:
len(hot_mat)

80

In [22]:
hot_names

['Wake',
 'Your',
 'You',
 'Do',
 'Take',
 'Keynes]',
 'What',
 'A',
 "Don't",
 'Heart',
 'It',
 'Too',
 'Need',
 'Got',
 'Like',
 'No',
 'Me',
 'Not',
 'See',
 'From',
 'Just',
 'Wanna',
 'At',
 'Be',
 'Of',
 'Love',
 'Le',
 'Rest',
 '[feat.',
 'Girl',
 'End',
 "You'll",
 'Head',
 'Can',
 'Out',
 'Go',
 'In',
 'Long',
 'More',
 'All',
 'Last',
 'Put',
 'If',
 'One',
 'Want',
 'The',
 'Name',
 'To',
 'With',
 'Am',
 'Know',
 'My',
 'Tired',
 'Let',
 "Ain't",
 'Make',
 'Day',
 'When',
 'Star',
 'Where',
 "I'll",
 'Mind',
 'On',
 'Milton',
 'World',
 'Mexico]',
 'Up',
 'Was',
 'For',
 'Waiting',
 'Never',
 'I',
 '[Live',
 'Life',
 'Is',
 'Little',
 'Way',
 "Can't",
 'Night',
 'Zenith]']

In [30]:
hots = [hot_names, hot_mat]
hots = np.array(hots)

In [31]:
hots

array([['Wake', 'Your', 'You', 'Do', 'Take', 'Keynes]', 'What', 'A',
        "Don't", 'Heart', 'It', 'Too', 'Need', 'Got', 'Like', 'No', 'Me',
        'Not', 'See', 'From', 'Just', 'Wanna', 'At', 'Be', 'Of', 'Love',
        'Le', 'Rest', '[feat.', 'Girl', 'End', "You'll", 'Head', 'Can',
        'Out', 'Go', 'In', 'Long', 'More', 'All', 'Last', 'Put', 'If',
        'One', 'Want', 'The', 'Name', 'To', 'With', 'Am', 'Know', 'My',
        'Tired', 'Let', "Ain't", 'Make', 'Day', 'When', 'Star', 'Where',
        "I'll", 'Mind', 'On', 'Milton', 'World', 'Mexico]', 'Up', 'Was',
        'For', 'Waiting', 'Never', 'I', '[Live', 'Life', 'Is', 'Little',
        'Way', "Can't", 'Night', 'Zenith]'],
       [list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

### Generate and save the chord plot

In [76]:
name = 'hotties2093_limit4.html'
# gen_html(hot_mat, hot_names, name)

In [63]:
name = 'notties2388_limit3.html'
# gen_html(not_mat, not_names, name)

### Potential names

Hotties

'hotties2093_limit4.html'
SONG TITLE - potential album art
"Rest The Day, Out All Night" - Kesha bathtub, Last Friday Night
"Waiting On A Star" - Twinkle twinkle
"You Ain't Got It" - No Scrubs
"Where You Go On My Heart" - celine dion
"The End Of The World" - it's the end of the world as we know
"Wake Me Up On The Way" - country roads
"One More 'NO'"
"You Can't Take Me"
"Never Too Long"
"Name My Heart Your Heart"


Notties

'notties1008_limit3.html'
"Can't I Love Me"
"Don't You Love Me"

'notties2388_limit3.html'
"I Don't Do It" - meatloaf
"Man In My Baby" - nirvana baby swimming
"Life Of A Fool" - lovefool
"Don't Let Me Down"
"That Girl That I Can't Be"
"Now You Are My Man" - crazy girlfriend face
"Your In My Man"

## STOP HERE
### Generate title data from base file

In [ ]:
# previously saved file csv file  
# matched.to_csv(r'../words_songs_matched.zip', index = False)

In [ ]:
# reading csv file  
# matched = pd.read_csv(r"../words_songs_matched.zip") 

In [ ]:
# check the data
print(matched.shape)
matched.head()

### Selective data drop

In [ ]:
# no longer used, caution INPLACE

dropcols = ['track_id', 'song_id', 'artist_id', 'duration']

# matched.drop(columns=dropcol, inplace=True)
# matched.head()

### Extract unique songs and save file

In [ ]:
# drop duplicates against song title
deduped = matched.drop_duplicates(subset="title", keep='first', inplace=False)
print(deduped.shape)
deduped.head()

In [ ]:
deduped['title'] = deduped['title'].apply(Clean_titles)
print(deduped.shape)
deduped.head()

In [ ]:
# deduped.to_csv(r"../Py code/titles_clean.zip", index=False)

## Data Review Only

In [ ]:
hot_bigrams

# for bigram in hot_bigrams:
#     current = bigram[0][1]
#     print('current: ' + current)
#     previous = bigram[0][0]
#     print('prev: ' + previous)
#     count = bigram[1]
#     print('count: ' + str(count))

In [ ]:
z = len(hot_bigrams)
for i in range(z):
    if hotbigrams[i][1] > 1:
        print(hotbigrams[i])

In [ ]:
# here is the trick part, need to rip all the edges which are very light weight

print('Hotties len: ' + str(len(hot_names)))
print('Hotties len: ' + str(len(np.unique(np.array(hot_names)))))

In [ ]:
hot_data['Too'].sum

In [ ]:
hot_data.index

In [ ]:
hot_data

### Test with sklearn vectorizer

In [ ]:
# test sklearn countvectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 1)) 
co_occurrences = bigram_vectorizer.fit_transform(unique_hotties)

In [ ]:
print('Printing sparse matrix:', co_occurrences)
print('Printing dense matrix', co_occurrences.todense())
sum_occ = np.sum(co_occurrences.todense(),axis=0)
print('Sum of word-word occurrences:', sum_occ)
print('Pretty printig of co_occurrences count:')
z = zip(bigram_vectorizer.get_feature_names(),np.array(sum_occ)[0].tolist())
print(z)

In [ ]:
df = pd.DataFrame(z)
print(df)